# Data Importing, Cleaning, Exporting
## Data Used
### Original Source
Data used comes from the below link. It is all police incident data documenting vehicular stops until July 15, 2025
https://data.cityofnewyork.us/Public-Safety/NYPD-Vehicle-Stop-Reports/hn9i-dwpr/about_data

### Postgre
Data is imported from a postgre server it was uploaded to, due to it being to large to interact with and check in excel. After Cleaning, it will be uploaded to a final table in Postgre for analysis

In [38]:
# imports
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
import math
import os

In [39]:
# Load environment variables
load_dotenv()

# Read them into variables
DB_USER = os.getenv("db_username")
DB_PASS = os.getenv("db_password")
DB_HOST = os.getenv("db_host")
DB_PORT = os.getenv("db_port")
DB_NAME = os.getenv("db_name")

# Build the SQLAlchemy connection string
connection_url = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [40]:
# Link Database
engine = create_engine(connection_url)

# Test Connection
with engine.connect() as conn:
    print("Connected to PostgreSQL")

Connected to PostgreSQL


In [41]:
# Query to take all data from table
query = "Select * From vehicle_stop_reports_raw;"

# Creating a dataframe to contain all table info
df = pd.read_sql(query, engine)

# Show table row count and column count, as well as top 5 rows
print(df.shape)
df.head()

(2039862, 19)


,evnt_key,occur_dt,occur_tm,cmd_cd,veh_seized_flg,veh_searched_flg,veh_search_consent_flg,veh_checkpoint_flg,force_used_flg,arrest_made_flg,summon_issued_flg,veh_category,rpted_age,sex_cd,race_desc,latitude,longitude,x_coord_cd,y_coord_cd
0,298749460,01/01/2025,14:16:02,7,false,false,(null),false,false,false,true,CAR/SUV,42,M,HISPANIC,,,,
1,298749748,01/01/2025,16:30:00,45,false,false,(null),false,false,false,false,CAR/SUV,28,M,HISPANIC,,,,
2,298750388,01/01/2025,21:25:00,110,false,true,N,false,false,true,false,CAR/SUV,41,M,BLACK,,,,
3,298750463,01/01/2025,23:15:00,114,false,false,(null),false,false,false,true,CAR/SUV,38,F,HISPANIC,,,,
4,298765275,01/02/2025,8:45:00,42,false,false,(null),false,false,false,false,CAR/SUV,61,M,BLACK,,,,


In [42]:
# Rename Columns to match Clean Table
df.rename(columns={
    "evnt_key": "event_key",
    "occur_dt": "date_occurred",
    "occur_tm": "time_occurred",
    "cmd_cd": "command_code",
    "veh_seized_flg": "vehicle_seized",
    "veh_searched_flg": "vehicle_searched",
    "veh_search_consent_flg": "vehicle_search_consent",
    "veh_checkpoint_flg": "vehicle_checkpoint",
    "force_used_flg": "force_used",
    "arrest_made_flg": "arrest_made",
    "summon_issued_flg": "summons_issued",
    "veh_category": "vehicle_category",
    "rpted_age": "reported_age",
    "sex_cd": "sex_code",
    "race_desc": "race_description",
    "latitude": "latitude",
    "longitude": "longitude",
    "x_coord_cd": "x_coordinate",
    "y_coord_cd": "y_coordinate",
}, inplace=True)

# Check top rows for change
df.head()

,event_key,date_occurred,time_occurred,command_code,vehicle_seized,vehicle_searched,vehicle_search_consent,vehicle_checkpoint,force_used,arrest_made,summons_issued,vehicle_category,reported_age,sex_code,race_description,latitude,longitude,x_coordinate,y_coordinate
0,298749460,01/01/2025,14:16:02,7,false,false,(null),false,false,false,true,CAR/SUV,42,M,HISPANIC,,,,
1,298749748,01/01/2025,16:30:00,45,false,false,(null),false,false,false,false,CAR/SUV,28,M,HISPANIC,,,,
2,298750388,01/01/2025,21:25:00,110,false,true,N,false,false,true,false,CAR/SUV,41,M,BLACK,,,,
3,298750463,01/01/2025,23:15:00,114,false,false,(null),false,false,false,true,CAR/SUV,38,F,HISPANIC,,,,
4,298765275,01/02/2025,8:45:00,42,false,false,(null),false,false,false,false,CAR/SUV,61,M,BLACK,,,,


In [43]:
# Replace all null values and empty strings with actual NaN
df = df.replace(['(null)', 'NULL', '', ' '], pd.NA)

# Check top rows for change
df.head()

,event_key,date_occurred,time_occurred,command_code,vehicle_seized,vehicle_searched,vehicle_search_consent,vehicle_checkpoint,force_used,arrest_made,summons_issued,vehicle_category,reported_age,sex_code,race_description,latitude,longitude,x_coordinate,y_coordinate
0,298749460,01/01/2025,14:16:02,7,false,false,<NA>,false,false,false,true,CAR/SUV,42,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
1,298749748,01/01/2025,16:30:00,45,false,false,<NA>,false,false,false,false,CAR/SUV,28,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
2,298750388,01/01/2025,21:25:00,110,false,true,N,false,false,true,false,CAR/SUV,41,M,BLACK,<NA>,<NA>,<NA>,<NA>
3,298750463,01/01/2025,23:15:00,114,false,false,<NA>,false,false,false,true,CAR/SUV,38,F,HISPANIC,<NA>,<NA>,<NA>,<NA>
4,298765275,01/02/2025,8:45:00,42,false,false,<NA>,false,false,false,false,CAR/SUV,61,M,BLACK,<NA>,<NA>,<NA>,<NA>


In [44]:
# Fix Date & Time columns
df['date_occurred'] = pd.to_datetime(df['date_occurred'], errors='coerce')
df['time_occurred'] = pd.to_datetime(df['time_occurred'], format='%H:%M:%S', errors='coerce').dt.time

# Check top rows for change
df.head()

,event_key,date_occurred,time_occurred,command_code,vehicle_seized,vehicle_searched,vehicle_search_consent,vehicle_checkpoint,force_used,arrest_made,summons_issued,vehicle_category,reported_age,sex_code,race_description,latitude,longitude,x_coordinate,y_coordinate
0,298749460,2025-01-01,14:16:02,7,false,false,<NA>,false,false,false,true,CAR/SUV,42,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
1,298749748,2025-01-01,16:30:00,45,false,false,<NA>,false,false,false,false,CAR/SUV,28,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
2,298750388,2025-01-01,21:25:00,110,false,true,N,false,false,true,false,CAR/SUV,41,M,BLACK,<NA>,<NA>,<NA>,<NA>
3,298750463,2025-01-01,23:15:00,114,false,false,<NA>,false,false,false,true,CAR/SUV,38,F,HISPANIC,<NA>,<NA>,<NA>,<NA>
4,298765275,2025-01-02,08:45:00,42,false,false,<NA>,false,false,false,false,CAR/SUV,61,M,BLACK,<NA>,<NA>,<NA>,<NA>


In [45]:
# Convert 'Y' / 'N' to boolean
def yn_to_bool(val):
    if pd.isna(val):
        return None
    return str(val).strip().upper() == 'Y'

df['vehicle_search_consent'] = df['vehicle_search_consent'].map(yn_to_bool)

# Check top rows for change
df.head()

,event_key,date_occurred,time_occurred,command_code,vehicle_seized,vehicle_searched,vehicle_search_consent,vehicle_checkpoint,force_used,arrest_made,summons_issued,vehicle_category,reported_age,sex_code,race_description,latitude,longitude,x_coordinate,y_coordinate
0,298749460,2025-01-01,14:16:02,7,false,false,None,false,false,false,true,CAR/SUV,42,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
1,298749748,2025-01-01,16:30:00,45,false,false,None,false,false,false,false,CAR/SUV,28,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
2,298750388,2025-01-01,21:25:00,110,false,true,False,false,false,true,false,CAR/SUV,41,M,BLACK,<NA>,<NA>,<NA>,<NA>
3,298750463,2025-01-01,23:15:00,114,false,false,None,false,false,false,true,CAR/SUV,38,F,HISPANIC,<NA>,<NA>,<NA>,<NA>
4,298765275,2025-01-02,08:45:00,42,false,false,None,false,false,false,false,CAR/SUV,61,M,BLACK,<NA>,<NA>,<NA>,<NA>


In [46]:
# Convert TRUE/FALSE strings to real booleans
for col in [
    'vehicle_seized', 'vehicle_searched', 'vehicle_checkpoint',
    'force_used', 'arrest_made', 'summons_issued'
]:
    df[col] = df[col].astype(str).str.upper().replace({
        'TRUE': True, 'FALSE': False, 'N': False, 'Y': True, 'nan': pd.NA
    })

# Check top rows for change
df.head()

C:\Users\spres\AppData\Local\Temp\ipykernel_8724\2112413223.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].astype(str).str.upper().replace({
C:\Users\spres\AppData\Local\Temp\ipykernel_8724\2112413223.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].astype(str).str.upper().replace({
C:\Users\spres\AppData\Local\Temp\ipykernel_8724\2112413223.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly

,event_key,date_occurred,time_occurred,command_code,vehicle_seized,vehicle_searched,vehicle_search_consent,vehicle_checkpoint,force_used,arrest_made,summons_issued,vehicle_category,reported_age,sex_code,race_description,latitude,longitude,x_coordinate,y_coordinate
0,298749460,2025-01-01,14:16:02,7,False,False,None,False,False,False,True,CAR/SUV,42,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
1,298749748,2025-01-01,16:30:00,45,False,False,None,False,False,False,False,CAR/SUV,28,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
2,298750388,2025-01-01,21:25:00,110,False,True,False,False,False,True,False,CAR/SUV,41,M,BLACK,<NA>,<NA>,<NA>,<NA>
3,298750463,2025-01-01,23:15:00,114,False,False,None,False,False,False,True,CAR/SUV,38,F,HISPANIC,<NA>,<NA>,<NA>,<NA>
4,298765275,2025-01-02,08:45:00,42,False,False,None,False,False,False,False,CAR/SUV,61,M,BLACK,<NA>,<NA>,<NA>,<NA>


In [47]:
# Handle 'UNKNOWN' args
df['reported_age'] = pd.to_numeric(df['reported_age'], errors='coerce')

# Check top rows for change
df.head()

,event_key,date_occurred,time_occurred,command_code,vehicle_seized,vehicle_searched,vehicle_search_consent,vehicle_checkpoint,force_used,arrest_made,summons_issued,vehicle_category,reported_age,sex_code,race_description,latitude,longitude,x_coordinate,y_coordinate
0,298749460,2025-01-01,14:16:02,7,False,False,None,False,False,False,True,CAR/SUV,42.0,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
1,298749748,2025-01-01,16:30:00,45,False,False,None,False,False,False,False,CAR/SUV,28.0,M,HISPANIC,<NA>,<NA>,<NA>,<NA>
2,298750388,2025-01-01,21:25:00,110,False,True,False,False,False,True,False,CAR/SUV,41.0,M,BLACK,<NA>,<NA>,<NA>,<NA>
3,298750463,2025-01-01,23:15:00,114,False,False,None,False,False,False,True,CAR/SUV,38.0,F,HISPANIC,<NA>,<NA>,<NA>,<NA>
4,298765275,2025-01-02,08:45:00,42,False,False,None,False,False,False,False,CAR/SUV,61.0,M,BLACK,<NA>,<NA>,<NA>,<NA>


In [48]:
# Convert numeric coordinates
for col in ['latitude', 'longitude', 'x_coordinate', 'y_coordinate']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check top rows for change
df.head()

,event_key,date_occurred,time_occurred,command_code,vehicle_seized,vehicle_searched,vehicle_search_consent,vehicle_checkpoint,force_used,arrest_made,summons_issued,vehicle_category,reported_age,sex_code,race_description,latitude,longitude,x_coordinate,y_coordinate
0,298749460,2025-01-01,14:16:02,7,False,False,None,False,False,False,True,CAR/SUV,42.0,M,HISPANIC,NaN,NaN,NaN,NaN
1,298749748,2025-01-01,16:30:00,45,False,False,None,False,False,False,False,CAR/SUV,28.0,M,HISPANIC,NaN,NaN,NaN,NaN
2,298750388,2025-01-01,21:25:00,110,False,True,False,False,False,True,False,CAR/SUV,41.0,M,BLACK,NaN,NaN,NaN,NaN
3,298750463,2025-01-01,23:15:00,114,False,False,None,False,False,False,True,CAR/SUV,38.0,F,HISPANIC,NaN,NaN,NaN,NaN
4,298765275,2025-01-02,08:45:00,42,False,False,None,False,False,False,False,CAR/SUV,61.0,M,BLACK,NaN,NaN,NaN,NaN


In [49]:
# Clean Strings
df['race_description'] = df['race_description'].str.upper().str.strip()
df['sex_code'] = df['sex_code'].str.upper().str.strip()
df['vehicle_category'] = df['vehicle_category'].str.upper().str.strip()

# Check top rows for change
df.head()

,event_key,date_occurred,time_occurred,command_code,vehicle_seized,vehicle_searched,vehicle_search_consent,vehicle_checkpoint,force_used,arrest_made,summons_issued,vehicle_category,reported_age,sex_code,race_description,latitude,longitude,x_coordinate,y_coordinate
0,298749460,2025-01-01,14:16:02,7,False,False,None,False,False,False,True,CAR/SUV,42.0,M,HISPANIC,NaN,NaN,NaN,NaN
1,298749748,2025-01-01,16:30:00,45,False,False,None,False,False,False,False,CAR/SUV,28.0,M,HISPANIC,NaN,NaN,NaN,NaN
2,298750388,2025-01-01,21:25:00,110,False,True,False,False,False,True,False,CAR/SUV,41.0,M,BLACK,NaN,NaN,NaN,NaN
3,298750463,2025-01-01,23:15:00,114,False,False,None,False,False,False,True,CAR/SUV,38.0,F,HISPANIC,NaN,NaN,NaN,NaN
4,298765275,2025-01-02,08:45:00,42,False,False,None,False,False,False,False,CAR/SUV,61.0,M,BLACK,NaN,NaN,NaN,NaN


In [50]:
# Push Back to SQL
df.to_sql(
    "nypd_vehicle_stop_reports",
    engine,
    if_exists="append",
    index=False,
    chunksize=10000,
    method="multi"
)

2039862